# Recognizing SIGN Language using Residual Networks

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

from test_utils import summary, comparator
import public_tests

%matplotlib inline

In [2]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    cashe = []

    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)

    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)

    return X

In [3]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

public_tests.identity_block_test(identity_block)

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.7123  192.7123  192.7123   96.85615]
  [ 96.85615  96.85615  96.85615  48.92808]]

 [[578.1369  578.1369  578.1369  290.56845]
  [290.56845 290.56845 290.56845 146.78423]]]
96.85615

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
All tests passed!


In [5]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)

    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [6]:
from outputs import convolutional_block_output1, convolutional_block_output2
np.random.seed(1)

X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
assert np.allclose(A.numpy(), convolutional_block_output1), "Wrong values when training=False."
print(A[0])

B = convolutional_block(X, f = 2, filters = [2, 4, 6], training=True)
assert np.allclose(B.numpy(), convolutional_block_output2), "Wrong values when training=True."

print('\033[92mAll tests passed!')


tf.Tensor(
[[[0.         0.66683805 0.         0.         0.888539   0.5274254 ]
  [0.         0.65053654 0.         0.         0.8959285  0.49965227]]

 [[0.         0.63120776 0.         0.         0.86362475 0.47643146]
  [0.         0.568832   0.         0.         0.8553412  0.417093  ]]], shape=(2, 2, 6), dtype=float32)
All tests passed!


**Expected value**

```
tf.Tensor(
[[[0.         0.66683817 0.         0.         0.88853896 0.5274254 ]
  [0.         0.65053666 0.         0.         0.89592844 0.49965227]]

 [[0.         0.6312079  0.         0.         0.8636247  0.47643146]
  [0.         0.5688321  0.         0.         0.85534114 0.41709304]]], shape=(2, 2, 6), dtype=float32)
```

In [7]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])

    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = AveragePooling2D((2, 2))(X)

    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)

    model = Model(inputs = X_input, outputs = X)

    return model

In [8]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 32, 32, 64)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization_20 (BatchN  (None, 32, 32, 64)  256         ['conv2d_20[0][0]']              
 ormalization)                                                                                

 conv2d_30 (Conv2D)             (None, 15, 15, 256)  16640       ['activation_26[0][0]']          
                                                                                                  
 batch_normalization_30 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_30[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_8 (Add)                    (None, 15, 15, 256)  0           ['activation_24[0][0]',          
                                                                  'batch_normalization_30[0][0]'] 
                                                                                                  
 activation_27 (Activation)     (None, 15, 15, 256)  0           ['add_8[0][0]']                  
                                                                                                  
 conv2d_31

                                                                  'batch_normalization_40[0][0]'] 
                                                                                                  
 activation_36 (Activation)     (None, 8, 8, 512)    0           ['add_11[0][0]']                 
                                                                                                  
 conv2d_41 (Conv2D)             (None, 8, 8, 128)    65664       ['activation_36[0][0]']          
                                                                                                  
 batch_normalization_41 (BatchN  (None, 8, 8, 128)   512         ['conv2d_41[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 8, 8, 128)    0           ['batch_normalization_41[0][0]'] 
          

 batch_normalization_51 (BatchN  (None, 4, 4, 256)   1024        ['conv2d_51[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_46 (Activation)     (None, 4, 4, 256)    0           ['batch_normalization_51[0][0]'] 
                                                                                                  
 conv2d_52 (Conv2D)             (None, 4, 4, 256)    590080      ['activation_46[0][0]']          
                                                                                                  
 batch_normalization_52 (BatchN  (None, 4, 4, 256)   1024        ['conv2d_52[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

                                                                                                  
 conv2d_62 (Conv2D)             (None, 4, 4, 1024)   263168      ['activation_56[0][0]']          
                                                                                                  
 batch_normalization_62 (BatchN  (None, 4, 4, 1024)  4096        ['conv2d_62[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_18 (Add)                   (None, 4, 4, 1024)   0           ['activation_54[0][0]',          
                                                                  'batch_normalization_62[0][0]'] 
                                                                                                  
 activation_57 (Activation)     (None, 4, 4, 1024)   0           ['add_18[0][0]']                 
          

 add_21 (Add)                   (None, 2, 2, 2048)   0           ['activation_63[0][0]',          
                                                                  'batch_normalization_72[0][0]'] 
                                                                                                  
 activation_66 (Activation)     (None, 2, 2, 2048)   0           ['add_21[0][0]']                 
                                                                                                  
 average_pooling2d (AveragePool  (None, 1, 1, 2048)  0           ['activation_66[0][0]']          
 ing2D)                                                                                           
                                                                                                  
 flatten (Flatten)              (None, 2048)         0           ['average_pooling2d[0][0]']      
                                                                                                  
 dense (De

In [9]:
from outputs import ResNet50_summary

model = ResNet50(input_shape = (64, 64, 3), classes = 6)

comparator(summary(model), ResNet50_summary)


All tests passed!


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [12]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
34/34 [==============================] - 69s 2s/step - loss: 2.1346 - accuracy: 0.4528
Epoch 2/10
34/34 [==============================] - 67s 2s/step - loss: 1.1318 - accuracy: 0.6370
Epoch 3/10
34/34 [==============================] - 62s 2s/step - loss: 0.9539 - accuracy: 0.7694
Epoch 4/10
34/34 [==============================] - 62s 2s/step - loss: 0.2830 - accuracy: 0.9000
Epoch 5/10
34/34 [==============================] - 62s 2s/step - loss: 0.2203 - accuracy: 0.9250
Epoch 6/10
34/34 [==============================] - 62s 2s/step - loss: 0.0987 - accuracy: 0.9667
Epoch 7/10
34/34 [==============================] - 62s 2s/step - loss: 0.1189 - accuracy: 0.9648
Epoch 8/10
34/34 [==============================] - 61s 2s/step - loss: 0.0860 - accuracy: 0.9731
Epoch 9/10
34/34 [==============================] - 62s 2s/step - loss: 0.1260 - accuracy: 0.9630
Epoch 10/10
34/34 [==============================] - 61s 2s/step - loss: 0.1179 - accuracy: 0.9583


In [13]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 2s 219ms/step - loss: 0.2917 - accuracy: 0.9250
Loss = 0.2917095720767975
Test Accuracy = 0.925000011920929


In [14]:
pre_trained_model = tf.keras.models.load_model('resnet50.h5')

In [15]:
preds = pre_trained_model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 2s 178ms/step - loss: 0.1596 - accuracy: 0.9500
Loss = 0.15958663821220398
Test Accuracy = 0.949999988079071


In [ ]:
%%javascript
IPython.notebook.save_checkpoint();
if (confirm("Clear memory?") == true)
{
    IPython.notebook.kernel.restart();
}

#### Test on My Own Image

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
imshow(img)
prediction = pre_trained_model.predict(x)
print("Class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ", prediction)
print("Class:", np.argmax(prediction))


In [ ]:
pre_trained_model.summary()

In [ ]:
%%javascript
IPython.notebook.save_checkpoint();
if (confirm("Clear memory?") == true)
{
    IPython.notebook.kernel.restart();
}